In [ ]:
import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

In [ ]:
env = gym.make("FrozenLake-v1", render_mode="human")
env.reset()

env.render()

In [ ]:
# observation space - states
print(env.observation_space)
print(env.observation_space.n)

# action space - (0) left, (1) down, (2) right, (3) up
print(env.action_space)
print(env.action_space.n)

# Frozen Lake - Stochastic movement

Note that if `is_slippery` is True (default), then the agent will move in the intended direction with a probability of 1/3, otherwise they will move in either *perpendicular* direction with an equal probability of 1/3 in both directions. Hence, an action `right` may result in the agent moving, `down` or `up` instead!

In [ ]:
env.reset()
for _ in range(10):
    # generate random action
    randomAction = env.action_space.sample()
    print(["left", "down", "right", "up"][randomAction])

    # Peform sampled action. The format of returnValue is (next_state, reward, terminated, truncated, info)
    next_state, reward, terminated, truncated, info = env.step(randomAction)
    print(next_state, reward, terminated, truncated, info)
    
    time.sleep(5)
    if terminated: 
        env.reset()
        print("Environment reset!")

In [ ]:

q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

In [ ]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [ ]:
rewards_all_episodes = []

# Q-learning algorithmm
for episode in range(num_episodes):
    state, _ = env.reset()

    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode):

        # Exploration-exploitation trade-off
        exploration_rate_treshold = random.uniform(0, 1)
        if exploration_rate_treshold > exploration_rate:
            action = np.argmax(q_table[state, :])
        else:
            action = env.action_space.sample()
        
        new_state, reward, done, info, _ = env.step(action)

        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :])) 

        state = new_state
        rewards_current_episode += reward
        if done == True:
            break

        # Exploration rate decay
        exploration_rate = min_exploration_rate + \
            (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)

        rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.array_split(np.array(rewards_all_episodes), num_episodes / 1000)
count = 1000
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes: 
    print(count, ": ", str(sum(r / 1000)))
    count += 1000

# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)